In [45]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import json 
import sys
#sys.path.append('../dataloaders/')
#sys.path.append('../utils/')
sys.path.append('../')
from tqdm import tqdm 

from dataloaders.dataloader import DataLoader 
import plotting
from utils.geometry import calc_distance
from utils.find_compatible_hits_dev import Find_Compatible_Hits_ModuleMap_Line_New


In [46]:
f = open("/home/lhv14/mapping_noMissing_noPtcut__noDoubles_1000.json") 

mappings = json.load(f)

In [47]:
dl = DataLoader() 
hits, allowed_pids = dl.load_data_trackml(0)

../dataloaders/dataloader.py:154: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  self.hits[['next_mod_id', 'connecting_layer_id']] = self.hits.groupby('particle_id')['discrete_module_id', 'unique_layer_id'].shift(-1)


In [48]:

success = 0 
hitsfound, tries = 0, 0 

#for pid in tqdm(np.unique(hits.particle_id)[:20]): 
for pid in tqdm(allowed_pids[:100]):
#for pid in rl_pids[: 
    p = hits[hits['particle_id']==pid]
    smallest_rs = p.groupby(['particle_id', 'unique_layer_id'])['r'].min()
    p = p.merge(smallest_rs)
    
    p[['prev_z', 'prev_r']] = p.groupby('particle_id')[['z', 'r']].shift(1)
    p['m'] = (p['r'] - p['prev_r'])/(p['z'] - p['prev_z'])
    p['b'] = p['r'] - p['m']*p['z']
    p[['next_r', 'next_z', 'next_hit_id']] = p.groupby('particle_id')[['r', 'z', 'hit_id']].shift(-1)

    #print(p)
    start_hit = p.iloc[0]
    next_start_hit = p.iloc[1]
    
    if start_hit.unique_layer_id == next_start_hit.unique_layer_id: 
        next_start_hit = p.iloc[2]
    track_reward = 0 

    #m = (next_start_hit.r - start_hit.r)/(next_start_hit.z - start_hit.z)
    #b = next_start_hit.r - m*next_start_hit.z
    #print(p)
    hitids = [] 
    prev_m = m 
    prev_b = b 
    for i in range(len(p)-2):
        #print(hit1)
        hit1 = p.iloc[i]
        hit2 = p.iloc[i+1]
        hit3 = p.iloc[i+2]
        

        m = hit2.m 
        b = hit2.b
#         m = (hit2.r - hit1.r)/(hit2.z - hit1.z)
#         b = hit2.r - m*hit2.z
        
#         if round(hit2.m, 2) != round(m, 2): 
#             print("not the same", hit2.m, m)
             
        try: 
            comp_mod = mappings[str(int(hit2.discrete_module_id))]
            #print("hit2 mod id", hit2.discrete_module_id)
        except: 
            #print("haven't seen this module", hit2.discrete_module_id)
            comp_mod = []
            
        
    
        if m > 0: 
            compy = hits[((hits['z'] > hit2.z) |  (hits['z'] == hit2.z))]
        else: 
            compy = hits[((hits['z'] < hit2.z) |  (hits['z'] == hit2.z))] 
        
        #comp_hits = compy[compy['discrete_module_id'].isin(comp_mod) & 
        #            (compy['r'] > hit2.r)]# & 
                    #(np.abs(hits['z']) > np.abs(hit2.z))]
            
        comp_hits = compy[compy['discrete_module_id'].isin(comp_mod)]
        #print("is in comp hits", hit3.hit_id in comp_hits.hit_id)
        #num_comp.append(len(comp_hits))
        
        distances = calc_distance(m,b, comp_hits.z, comp_hits.r)
        num_close = 10

        if len(comp_hits) > num_close: 
            idx = np.argpartition(distances, range(num_close))
            final = comp_hits.iloc[idx.values][:num_close]
        else: 
            final = comp_hits 
        
 
        
        #hitids.append(final.hit_id.values)
        if hit3.hit_id in final.hit_id.values: 
            hitsfound += 1 

        tries +=1 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 22.39it/s]


In [49]:
hitsfound/tries

0.9426111908177905

In [50]:
comp = Find_Compatible_Hits_ModuleMap_Line_New(hits)

success = 0 
hitsfound, tries = 0, 0 

#for pid in tqdm(np.unique(hits.particle_id)[:20]): 
for pid in tqdm(allowed_pids[:100]):
#for pid in rl_pids[: 
    p = hits[hits['particle_id']==pid]
    smallest_rs = p.groupby(['particle_id', 'unique_layer_id'])['r'].min()
    p = p.merge(smallest_rs)
    
    p[['prev_z', 'prev_r']] = p.groupby('particle_id')[['z', 'r']].shift(1)
    p['m'] = (p['r'] - p['prev_r'])/(p['z'] - p['prev_z'])
    p['b'] = p['r'] - p['m']*p['z']
    p[['next_r', 'next_z', 'next_hit_id']] = p.groupby('particle_id')[['r', 'z', 'hit_id']].shift(-1)

    #print(p)
    start_hit = p.iloc[0]
    next_start_hit = p.iloc[1]
    
    if start_hit.unique_layer_id == next_start_hit.unique_layer_id: 
        next_start_hit = p.iloc[2]
    track_reward = 0 

    #m = (next_start_hit.r - start_hit.r)/(next_start_hit.z - start_hit.z)
    #b = next_start_hit.r - m*next_start_hit.z
    #print(p)
    hitids = [] 
    prev_m = m 
    prev_b = b 
    for i in range(len(p)-2):
        #print(hit1)
        hit1 = p.iloc[i]
        hit2 = p.iloc[i+1]
        hit3 = p.iloc[i+2]
        

        m = hit2.m 
        b = hit2.b

        final = comp.get_comp_hits_notrow(hit2.z, hit2.r, hit2.m, hit2.b, num_close)[0]

 
        
        #hitids.append(final.hit_id.values)
        if hit3.hit_id in final.hit_id.values: 
            hitsfound += 1 

        tries +=1 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.00it/s]


In [51]:
hitsfound/tries

0.9497847919655668

In [16]:
p = hits[hits['particle_id']==allowed_pids[0]]
smallest_rs = p.groupby(['particle_id', 'unique_layer_id'])['r'].min()
p = p.merge(smallest_rs)

In [37]:
for i in range(len(p)-2):

    hit1 = p.iloc[i]
    hit2 = p.iloc[i+1]
    hit3 = p.iloc[i+2]
    m = (hit2.r - hit1.r)/(hit2.z - hit1.z)
    b = hit2.r - m*hit2.z

    print(m, b, hit2.m)

-0.23950683071783405 0.750442398761475 nan
-0.2377907355975841 0.8287773320276663 -0.2395068258047104
-0.2360624366522077 0.9321296076425867 -0.23779073357582092
-0.2344696573208979 1.0433056098287885 -0.2360624372959137
-0.2318507659338456 1.3232651031007912 -0.23446965217590332
-0.2298785618373326 1.562986517050632 -0.2318507730960846
-0.2158539613773006 3.6603655586481025 -0.22987855970859528
-0.2081571306501116 5.046949567173478 -0.2158539593219757
-0.1926426331204022 8.384893615997477 -0.20815713703632355


In [35]:
p[['hit_id', 'm', 'b']]

,hit_id,m,b
0,20687,NaN,NaN
1,25681,NaN,NaN
2,12045,-0.239507,0.750443
3,9681,-0.237791,0.828777
4,54171,-0.236062,0.932129
5,53585,-0.234470,1.043306
6,52485,-0.231851,1.323265
7,51394,-0.229879,1.562988
8,50350,-0.215854,3.660366
9,49316,-0.208157,5.046947


In [19]:
p[['prev_z', 'prev_r']] = p.groupby('particle_id')[['z', 'r']].shift(1)
p[['prev_prev_z', 'prev_prev_r']] = p.groupby('particle_id')[['prev_z', 'prev_r']].shift(1)
p['m'] = (p['prev_r'] - p['prev_prev_r'])/(p['prev_z'] - p['prev_prev_z'])
p['b'] = p['prev_r'] - p['m']*p['prev_z']
p[['next_r', 'next_z', 'next_hit_id']] = p.groupby('particle_id')[['r', 'z', 'hit_id']].shift(-1)


In [24]:
p[['hit_id', 'm', 'b']].iloc[2].b

-0.7948365211486816